In [14]:
# -*- coding: utf-8 -*-
import logging
import jieba
from opencc import OpenCC 
import pandas as pd
from pypinyin import pinyin,slug
import time

# to count in wiki
import sys
#import time
import math
from collections import Counter
import multiprocessing as mp
import threading as td
#import pandas as pd
import numpy as np

In [6]:
### file path of jieba dictionary
index = ord('A')
filePath = "C:/Users/miplab01/Desktop/jupyter notebook files/Bruce-Willis/jieba_letter/"
fileName = "jieba_letter_" + chr(index) + ".csv"

In [7]:
### find all substring of input
def indexes(seq, start=0):
    return (i for i,_ in enumerate(seq, start=start))

def gen_all_substrings(s):
    return (s[i:j] for i in indexes(s) for j in indexes(s[i:], i+1))

def get_all_substrings(string):
    return list(gen_all_substrings(string))

inputStr = "既生瑜"
input_pinyin = slug(inputStr,style = 8)

input_pinyin_lst = [] # input_pinyin_lst : save all substring of input pinyin
input_xieyin_lst = [] # input_xieyin_lst : save all item in jieba dictionary that has the same pinyin with the substring of input
label = [] # mark the input_xieyin_lst to know which word's xieyin is in this set
tmp = []

# get input_pinyin in string type
for idx,item in enumerate(get_all_substrings(inputStr)):
    input_pinyin_lst.append(slug(item,style = 8))

# find all xieyin in jieba dictionary and save in the correct list
for idx,item in enumerate(input_pinyin_lst):
    tmp = []
    label = []
    for idx_in,item_in in enumerate(input_pinyin.split("-")):
        if item.split("-")[0] == item_in:
            label.append(idx_in+1)
            break;
    for i in range(len(item.split("-"))):
        if len(label) < len(item.split("-")):
            label.append(label[len(label)-1] + 1)
    tmp.append(label)
    index = ord(slug(item,style = 4)[0]) - 32
    fileName = "jieba_letter_" + chr(index) + ".csv"
    df = pd.read_csv(filePath + fileName)
    for idx_df,py_df in enumerate(df.pinyin):
        if py_df == item:
            if df.frequency[idx_df] >= 600:
                tmp.append(df.loc[idx_df])
    input_xieyin_lst.append(tmp)

In [8]:
print("input\t: ", inputStr, "\n拼音\t: ", input_pinyin)
print("sub拼音\t:", input_pinyin_lst)

input	:  既生瑜 
拼音	:  ji4-sheng1-yu2
sub拼音	: ['ji4', 'ji4-sheng1', 'ji4-sheng1-yu2', 'sheng1', 'sheng1-yu2', 'yu2']


In [9]:
print(input_xieyin_lst[0][0])
print(input_xieyin_lst[0][1])

[1]
chinese         計
frequency    4965
pinyin        ji4
posseg          n
Name: 1683, dtype: object


In [12]:
### combine the substrings
def part(num):
    answer = set()
    answer.add((num, ))
    for x in range(1, num):
        for y in part(num-x):
            answer.add((x, ) + y)
    return answer

permu = part(len(inputStr))
record_b = 0 #record nth b
start_idx = 1 #start from which word, b represent length
tmp_chinese = []
tmp_pseg = []
tmp_next = []
tmp_next_pseg = []
tmp_label = []
xieyin = []
xieyin_pseg = []
noMatch = 0

for a in permu:
    print(a)
    tmp_label = []
    start_idx = 1
    record_b = 0
    tmp_chinese = []
    tmp_pseg = []
    tmp_next = []
    tmp_next_pseg = []
    noMatch = 0
    
    for b in a:
        tmp_label = []
        record_b += 1
        if record_b == 1 :
            if b == 1:
                tmp_label.append(record_b)
            else:
                for i in range(b):
                    tmp_label.append(start_idx + i)
        elif record_b > 1:
            for i in range(b):
                tmp_label.append(start_idx + i)        
        start_idx += b 
        
        for i in range(len(input_xieyin_lst)):
            if input_xieyin_lst[i][0] == tmp_label:
                if len(input_xieyin_lst[i]) > 1:
                    ;
                else:
                    noMatch = 1
                    print("no matched string")
                    break
        
        for i in range(len(input_xieyin_lst)):
            if noMatch == 0:
                if input_xieyin_lst[i][0] == tmp_label:
                    if len(input_xieyin_lst[i]) > 1:
                        if len(tmp_chinese) == 0:
                            for idx_xy,item_xy in enumerate(input_xieyin_lst[i]):
                                if idx_xy == 0:
                                    continue
                                tmp_chinese.append(item_xy.chinese)
                                tmp_pseg.append(item_xy.posseg)
                        else:
                            for idx_tmp, item_tmp in enumerate(tmp_chinese):
                                for idx_xy,item_xy in enumerate(input_xieyin_lst[i]):
                                    if idx_xy == 0:
                                        continue
                                    tmp_next.append(tmp_chinese[idx_tmp] + item_xy.chinese)
                                    tmp_next_pseg.append(tmp_pseg[idx_tmp] + '+' + item_xy.posseg)                                    
                            tmp_chinese = tmp_next
                            tmp_pseg = tmp_next_pseg
                            tmp_next = []
                            tmp_next_pseg = []
    
    for item in tmp_chinese:
        xieyin.append(item)
    for item in tmp_pseg:
        xieyin_pseg.append(item)
    print("len = ",len(xieyin))
    print('--------')

xieyin_df = pd.DataFrame({'xieyin': xieyin, 'xieyin_pseg': xieyin_pseg})
#xieyin_df = pd.DataFrame()
#xieyin_df['xieyin'] = xieyin
#xieyin_df['xieyin_pseg'] = xieyin_pseg
xieyin_df.head(20)

(1, 2)
len =  13
--------
(1, 1, 1)
len =  403
--------
(3,)
no matched string
len =  403
--------
(2, 1)
len =  413
--------


,xieyin,xieyin_pseg
0,計生於,n+v
1,劑生於,zg+v
2,寄生於,v+v
3,際生於,ng+v
4,技生於,n+v
5,紀生於,nr+v
6,繼生於,v+v
7,記生於,n+v
8,祭生於,v+v
9,季生於,n+v


In [15]:
# define thread class

class Count_in_file_thread(td.Thread):
	def __init__(self, string):
		super(Count_in_file_thread,  self ).__init__()
		self.string = string
		self.count = 0

	def read_in_chunks(self, chunk_size = 4096):
		while True:
			data = self.file_ob.read(chunk_size)
			if not data:
				break
			yield data

	def get_count(self):
		return self.count

	def get_string(self):
		return self.string

	def run(self):
		self.file_ob = open(wikifile_path, 'r', encoding='utf-8')
		for piece in self.read_in_chunks():
			self.count += piece.count(self.string)
	
# function

def count_in_file(wikifile_path, word_df):

	# initialize

	word_df['count'] = np.zeros(len(word_df.index))
	word_list = word_df['xieyin'].tolist()

	thread_num = 10
	thread_list = []

	# divide list

	s_list = []
	got_uncount = 0

	for index, row in word_df.iterrows():
		if( (index+1) % thread_num == 0 ):
			# create threads
			for w in s_list:
				thread_list.append( Count_in_file_thread(w) )

			for t in thread_list:
				t.start()

			for t in thread_list:
				t.join()

			for t in thread_list:
				word_df.loc[word_df['xieyin'] == t.get_string(), 'count'] = t.get_count()

			got_uncount = 0
			thread_list[:] = []
			s_list[:] = []
		
		s_list.append(row['xieyin'])
		got_uncount = 1

	
	if( got_uncount == 1 ):
		for w in s_list:
			thread_list.append( Count_in_file_thread(w) )

		for t in thread_list:
			t.start()

		for t in thread_list:
			t.join()

		for t in thread_list:
			word_df.loc[word_df['xieyin'] == t.get_string(), 'count'] = t.get_count()

	# sort by count

	word_df = word_df.sort_values(by=['count'])
	return word_df


def time_since(since):
	s = time.time() - since
	m = math.floor(s/60)
	s -= m*60
	return '%dm %ds' % (m, s)

In [16]:
wikifile_path = 'wiki_texts_tw.txt'
start_time = time.time()

word_in_wiki = count_in_file(wikifile_path, xieyin_df)
print(time_since(start_time))

35m 38s


In [17]:
word_in_wiki.head(10)

,xieyin,xieyin_pseg,count
0,計生於,n+v,0.0
277,祭聲魚,v+n+n,0.0
276,祭聲餘,v+n+m,0.0
275,祭聲於,v+n+p,0.0
274,祭聲渝,v+n+j,0.0
273,祭聲輿,v+n+n,0.0
272,祭升嬩,v+zg+zg,0.0
271,祭升逾,v+zg+vg,0.0
270,祭升漁,v+zg+n,0.0
269,祭升瑜,v+zg+nr,0.0


In [18]:
word_in_wiki_tmp = word_in_wiki.copy()
for i,row in word_in_wiki_tmp.iterrows():
    if row["count"] != 0:
        print(row["xieyin"],row["xieyin_pseg"],row["count"])

濟生於 j+v 1.0
既生於 c+v 1.0
紀生於 nr+v 1.0
濟生於 j+vn+p 1.0
既生於 c+vn+p 1.0
紀生於 nr+vn+p 1.0
繼生於 v+vn+p 2.0
繼生於 v+v 2.0
既生瑜 c+vn+nr 6.0
寄生於 v+vn+p 161.0
寄生於 v+p 161.0
寄生於 v+v 161.0
